In [1]:
import pymorphy2
import torch

import numpy as np
import pandas as pd

from transformers import AutoTokenizer, AutoModel

In [2]:
class PymorphyMorpher:    
    def __init__(self):
        self.morpher = pymorphy2.MorphAnalyzer()
        self.cash = {}
        
    def normal_form(self, word):
        if word in self.cash:
            lemma = self.cash[word]
        else:
                lemma = self.morpher.parse(word)[0].normal_form
                self.cash[word] = lemma
        return lemma

## Данные

In [3]:
train_df = pd.read_csv('../russe-wsi-kit/data/main/bts-rnc/train.csv', delimiter='\t')

In [4]:
train_df

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,1,балка,1,NaN,90-94,"маленькой комнаты. Он был очень высок, наклони..."
1,2,балка,1,NaN,69-73,Пантюхин в Склифе сейчас. Он выползти на улицу...
2,3,балка,1,NaN,115-121,равнозначно обеспечивает и меланхоличную езду....
3,4,балка,1,NaN,85-90,"верхняя часть закрыта, замкнута, многократно о..."
4,5,балка,1,NaN,66-70,"по телевизору: наши гол забили, я вскочил от р..."
...,...,...,...,...,...,...
3486,3487,штамп,4,NaN,81-86,"дурак, но партию свою отрабатывал точно, по ус..."
3487,3488,штамп,4,NaN,85-90,"дело... получается, мыслить и выражать свое мн..."
3488,3489,штамп,4,NaN,71-77,", что актер должен иметь пять штампов-клише ""п..."
3489,3490,штамп,4,NaN,107-112,сегодняшний день в системе негосударственного ...


## BERT

In [5]:
morph = PymorphyMorpher()

In [12]:
def return_location(target, tokens):
    current = ''
    current_indices = []
    for i, token in enumerate(tokens):
        if token[:2] == '##':
            current += token[2:]
            current_indices.append(i)
        else:
            current = token
            current_indices = [i]
        current_lemma = morph.normal_form(current)
        if i < len(tokens)-1:
            next = tokens[i+1]
            if next[:2] != '##':
                if current_lemma == target:
                    return current_indices
        else:
            if current_lemma == target:
                return current_indices
    return 'not found'

In [13]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
print(device)
model = model.to(device)

cpu


In [15]:
train_df['tokens'] = train_df['context'].apply(tokenizer.tokenize)

In [16]:
%%time
train_df['word_location'] = train_df.apply(lambda x: return_location(x['word'], x['tokens']), axis=1)

CPU times: user 4.49 s, sys: 11.8 ms, total: 4.5 s
Wall time: 4.51 s


In [22]:
train_df[train_df['word_location'] == 'not found']

,context_id,word,gold_sense_id,predict_sense_id,positions,context,tokens,word_location
1023,1024,курица,1,NaN,99-101,", смешивали с другим вирусом-родителем (необлу...","[,, см, ##еши, ##вали, с, другим, ви, ##рус, #...",not found
1029,1030,курица,1,NaN,84-86,"звучала так: ""Баба Дуня, молодец. Своих кур, к...","[зв, ##учал, ##а, так, :, "", Ба, ##ба, Д, ##ун...",not found
1030,1031,курица,1,NaN,72-75,"потом в гнездо влетали, понеже они при влетани...","[потом, в, г, ##не, ##здо, в, ##лета, ##ли, ,,...",not found
1033,1034,курица,1,NaN,77-79,"и чечевицы. Яблоки, в сем ветру пересаженные, ...","[и, че, ##чев, ##ицы, ., Я, ##бло, ##ки, ,, в,...",not found
1036,1037,курица,1,NaN,81-84,жертвами птичьего гриппа становятся местные пт...,"[жертв, ##ами, п, ##ти, ##чь, ##его, гри, ##п,...",not found
...,...,...,...,...,...,...,...,...
3118,3119,рысь,2,NaN,80-83,же мягка и ленива. Но совсем по-другому. Ее мя...,"[же, м, ##яг, ##ка, и, ле, ##нив, ##а, ., Но, ...",not found
3119,3120,рысь,2,NaN,81-84,"созвездии Овна. Работе с животными, многие из ...","[со, ##зв, ##езд, ##ии, О, ##вна, ., Р, ##аб, ...",not found
3120,3121,рысь,2,NaN,77-80,". Они называли его «турусунь» и уверяли, что п...","[., Они, на, ##зывали, его, «, тур, ##усу, ##н...",not found
3123,3124,рысь,2,NaN,61-64,"тут было волков и лисиц, белок и зайцев, водил...","[тут, было, во, ##лков, и, ли, ##си, ##ц, ,, б...",not found


In [25]:
train_df[train_df['word_location'] == 'not found'].to_excel('bad_words.xlsx')

In [23]:
set(train_df[train_df['word_location'] == 'not found']['word'])

{'курица', 'лавка', 'лайка', 'лира', 'рысь'}

In [34]:
word = 'лайки'

In [35]:
morph.normal_form(word)

'лайк'

In [45]:
morph.morpher.parse('лайка')[0].lexeme

[Parse(word='лайк', tag=OpencorporaTag('NOUN,inan,masc sing,nomn'), normal_form='лайк', score=1.0, methods_stack=((DictionaryAnalyzer(), 'лайк', 19, 0),)),
 Parse(word='лайка', tag=OpencorporaTag('NOUN,inan,masc sing,gent'), normal_form='лайк', score=1.0, methods_stack=((DictionaryAnalyzer(), 'лайка', 19, 1),)),
 Parse(word='лайку', tag=OpencorporaTag('NOUN,inan,masc sing,datv'), normal_form='лайк', score=1.0, methods_stack=((DictionaryAnalyzer(), 'лайку', 19, 2),)),
 Parse(word='лайк', tag=OpencorporaTag('NOUN,inan,masc sing,accs'), normal_form='лайк', score=1.0, methods_stack=((DictionaryAnalyzer(), 'лайк', 19, 3),)),
 Parse(word='лайком', tag=OpencorporaTag('NOUN,inan,masc sing,ablt'), normal_form='лайк', score=1.0, methods_stack=((DictionaryAnalyzer(), 'лайком', 19, 4),)),
 Parse(word='лайке', tag=OpencorporaTag('NOUN,inan,masc sing,loct'), normal_form='лайк', score=1.0, methods_stack=((DictionaryAnalyzer(), 'лайке', 19, 5),)),
 Parse(word='лайки', tag=OpencorporaTag('NOUN,inan,mas

In [42]:
morph.morpher.parse('лайка')[1].lexeme

[Parse(word='лайка', tag=OpencorporaTag('NOUN,anim,femn sing,nomn'), normal_form='лайка', score=1.0, methods_stack=((DictionaryAnalyzer(), 'лайка', 90, 0),)),
 Parse(word='лайки', tag=OpencorporaTag('NOUN,anim,femn sing,gent'), normal_form='лайка', score=1.0, methods_stack=((DictionaryAnalyzer(), 'лайки', 90, 1),)),
 Parse(word='лайке', tag=OpencorporaTag('NOUN,anim,femn sing,datv'), normal_form='лайка', score=1.0, methods_stack=((DictionaryAnalyzer(), 'лайке', 90, 2),)),
 Parse(word='лайку', tag=OpencorporaTag('NOUN,anim,femn sing,accs'), normal_form='лайка', score=1.0, methods_stack=((DictionaryAnalyzer(), 'лайку', 90, 3),)),
 Parse(word='лайкой', tag=OpencorporaTag('NOUN,anim,femn sing,ablt'), normal_form='лайка', score=1.0, methods_stack=((DictionaryAnalyzer(), 'лайкой', 90, 4),)),
 Parse(word='лайкою', tag=OpencorporaTag('NOUN,anim,femn sing,ablt,V-oy'), normal_form='лайка', score=1.0, methods_stack=((DictionaryAnalyzer(), 'лайкою', 90, 5),)),
 Parse(word='лайке', tag=OpencorporaT

In [43]:
morph.morpher.parse('лайка')[2].lexeme

[Parse(word='лайка', tag=OpencorporaTag('NOUN,inan,femn sing,nomn'), normal_form='лайка', score=1.0, methods_stack=((DictionaryAnalyzer(), 'лайка', 146, 0),)),
 Parse(word='лайки', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='лайка', score=1.0, methods_stack=((DictionaryAnalyzer(), 'лайки', 146, 1),)),
 Parse(word='лайке', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='лайка', score=1.0, methods_stack=((DictionaryAnalyzer(), 'лайке', 146, 2),)),
 Parse(word='лайку', tag=OpencorporaTag('NOUN,inan,femn sing,accs'), normal_form='лайка', score=1.0, methods_stack=((DictionaryAnalyzer(), 'лайку', 146, 3),)),
 Parse(word='лайкой', tag=OpencorporaTag('NOUN,inan,femn sing,ablt'), normal_form='лайка', score=1.0, methods_stack=((DictionaryAnalyzer(), 'лайкой', 146, 4),)),
 Parse(word='лайкою', tag=OpencorporaTag('NOUN,inan,femn sing,ablt,V-oy'), normal_form='лайка', score=1.0, methods_stack=((DictionaryAnalyzer(), 'лайкою', 146, 5),)),
 Parse(word='лайке', tag=Openco

In [ ]:
with torch.no_grad():
    model_output = model(**{k: v.to(model.device) for k, v in t.items()})
embeddings = model_output.last_hidden_state[:, :]
embeddings = torch.nn.functional.normalize(embeddings)

In [ ]:
embeddings = embeddings.squeeze()

In [ ]:
embeddings[1].shape

In [ ]:
tokenizer.convert_tokens_to_ids(tokenized_text)